In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=e39ef64fc572e2bc535a32a77d4040ffc3f420fea69d8155062d2f8eb186c050
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [20]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Road Accident').getOrCreate()

In [21]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
df1 = spark.sparkContext.textFile('drive/My Drive/ABD/Road_accidents.csv')

In [23]:
df1 = spark.read.csv('drive/My Drive/ABD/Road_accidents.csv',header =True,inferSchema =True)
df1.show()
df1.printSchema()

+--------------+----+-------+--------+-----+-----+----+----+----+------+---------+-------+--------+--------+
|      STATE/UT|YEAR|JANUARY|FEBRUARY|MARCH|APRIL| MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER|
+--------------+----+-------+--------+-----+-----+----+----+----+------+---------+-------+--------+--------+
| A & N Islands|2001|      8|      23|   15|   15|  14|  19|  14|    19|        7|     12|      13|      22|
| A & N Islands|2002|     12|      10|   14|   16|  10|   7|  16|    11|       23|     21|      11|      17|
| A & N Islands|2003|     19|      13|   15|   13|  13|  12|   8|    16|       17|     25|      14|      15|
| A & N Islands|2004|     21|      14|   22|   17|  13|  18|  16|    19|       16|     20|      15|      24|
| A & N Islands|2005|     19|      21|   22|   17|  13|  19|  21|    14|       15|     19|      10|      16|
| A & N Islands|2006|     21|      13|    4|   22|   9|  14|  12|    14|        8|     14|       6|      18|
| A & N Islands|200

In [24]:
df2 = spark.sparkContext.textFile('drive/My Drive/ABD/Road_accidents_time.csv')

In [25]:
df2 = spark.read.csv('drive/My Drive/ABD/Road_accidents_time.csv',header = True, inferSchema = True)
df2.show()
df2.printSchema()

+--------------+----+----------------+----------------+-------------+--------------+---------------+---------------+-----------------+-----------------+
|      STATE/UT|YEAR|0-3 hrs. (Night)|3-6 hrs. (Night)|6-9 hrs (Day)|9-12 hrs (Day)|12-15 hrs (Day)|15-18 hrs (Day)|18-21 hrs (Night)|21-24 hrs (Night)|
+--------------+----+----------------+----------------+-------------+--------------+---------------+---------------+-----------------+-----------------+
| A & N Islands|2001|               2|               6|           29|            40|             39|             40|               18|                7|
| A & N Islands|2002|               2|               6|           22|            41|             33|             33|               23|                8|
| A & N Islands|2003|               2|               8|           31|            35|             28|             36|               25|               15|
| A & N Islands|2004|               2|               5|           29|            4

In [30]:
#1 Add column 'Total', giving sum of accidents in each month .
total = df1.withColumn('Sum',(col('JANUARY')+col('FEBRUARY')+col('MARCH')+col('APRIL')+col( 'MAY')+col('JUNE')+col('JULY')+col('AUGUST')+col('SEPTEMBER')+col('OCTOBER')+col('NOVEMBER')+col('DECEMBER')))
total.show()

+--------------+----+-------+--------+-----+-----+----+----+----+------+---------+-------+--------+--------+-----+
|      STATE/UT|YEAR|JANUARY|FEBRUARY|MARCH|APRIL| MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER|  Sum|
+--------------+----+-------+--------+-----+-----+----+----+----+------+---------+-------+--------+--------+-----+
| A & N Islands|2001|      8|      23|   15|   15|  14|  19|  14|    19|        7|     12|      13|      22|  181|
| A & N Islands|2002|     12|      10|   14|   16|  10|   7|  16|    11|       23|     21|      11|      17|  168|
| A & N Islands|2003|     19|      13|   15|   13|  13|  12|   8|    16|       17|     25|      14|      15|  180|
| A & N Islands|2004|     21|      14|   22|   17|  13|  18|  16|    19|       16|     20|      15|      24|  215|
| A & N Islands|2005|     19|      21|   22|   17|  13|  19|  21|    14|       15|     19|      10|      16|  206|
| A & N Islands|2006|     21|      13|    4|   22|   9|  14|  12|    14|        

In [33]:
# 2. Which state has highest number of accidents in year 2013?
acc = total.select('STATE/UT','YEAR','Sum').filter(col('year')==2013)
acc.where(col('Sum') == acc.agg({'Sum':'max'}).collect()[0][0]).show()


+----------+----+-----+
|  STATE/UT|YEAR|  Sum|
+----------+----+-----+
|Tamil Nadu|2013|66238|
+----------+----+-----+



In [34]:
#Find the average monthly accidents for each state.
avg = df1.withColumn('Avg',(col('JANUARY')+col('FEBRUARY')+col('MARCH')+col('APRIL')+col( 'MAY')+col('JUNE')+col('JULY')+col('AUGUST')+col('SEPTEMBER')+col('OCTOBER')+col('NOVEMBER')+col('DECEMBER'))/2)
avg.show()

+--------------+----+-------+--------+-----+-----+----+----+----+------+---------+-------+--------+--------+-------+
|      STATE/UT|YEAR|JANUARY|FEBRUARY|MARCH|APRIL| MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER|    Avg|
+--------------+----+-------+--------+-----+-----+----+----+----+------+---------+-------+--------+--------+-------+
| A & N Islands|2001|      8|      23|   15|   15|  14|  19|  14|    19|        7|     12|      13|      22|   90.5|
| A & N Islands|2002|     12|      10|   14|   16|  10|   7|  16|    11|       23|     21|      11|      17|   84.0|
| A & N Islands|2003|     19|      13|   15|   13|  13|  12|   8|    16|       17|     25|      14|      15|   90.0|
| A & N Islands|2004|     21|      14|   22|   17|  13|  18|  16|    19|       16|     20|      15|      24|  107.5|
| A & N Islands|2005|     19|      21|   22|   17|  13|  19|  21|    14|       15|     19|      10|      16|  103.0|
| A & N Islands|2006|     21|      13|    4|   22|   9|  14|  12

In [36]:
#4 Which month has highest accidents?
High_acc = avg.select('STATE/UT','YEAR','Avg').show()

+--------------+----+-------+
|      STATE/UT|YEAR|    Avg|
+--------------+----+-------+
| A & N Islands|2001|   90.5|
| A & N Islands|2002|   84.0|
| A & N Islands|2003|   90.0|
| A & N Islands|2004|  107.5|
| A & N Islands|2005|  103.0|
| A & N Islands|2006|   77.5|
| A & N Islands|2007|   76.0|
| A & N Islands|2008|   95.5|
| A & N Islands|2009|  135.5|
| A & N Islands|2010|  142.5|
| A & N Islands|2011|  117.5|
| A & N Islands|2012|  118.0|
| A & N Islands|2013|  100.0|
| A & N Islands|2014|  109.0|
|Andhra Pradesh|2001|13594.0|
|Andhra Pradesh|2002|16288.5|
|Andhra Pradesh|2003|17268.5|
|Andhra Pradesh|2004|18539.0|
|Andhra Pradesh|2005|18644.5|
|Andhra Pradesh|2006|20661.5|
+--------------+----+-------+
only showing top 20 rows



In [38]:
df2.show()

+--------------+----+----------------+----------------+-------------+--------------+---------------+---------------+-----------------+-----------------+
|      STATE/UT|YEAR|0-3 hrs. (Night)|3-6 hrs. (Night)|6-9 hrs (Day)|9-12 hrs (Day)|12-15 hrs (Day)|15-18 hrs (Day)|18-21 hrs (Night)|21-24 hrs (Night)|
+--------------+----+----------------+----------------+-------------+--------------+---------------+---------------+-----------------+-----------------+
| A & N Islands|2001|               2|               6|           29|            40|             39|             40|               18|                7|
| A & N Islands|2002|               2|               6|           22|            41|             33|             33|               23|                8|
| A & N Islands|2003|               2|               8|           31|            35|             28|             36|               25|               15|
| A & N Islands|2004|               2|               5|           29|            4

In [41]:
#5 Which time slot (like 0-3, 3-6 etc) has more accidents? What about 'Karnataka' state?
time = df2.withColumnRenamed('0-3 hrs. (Night)','Till 3 (night)')
time.filter(col('STATE/UT') == 'Karnataka').show()

+---------+----+--------------+----------------+-------------+--------------+---------------+---------------+-----------------+-----------------+
| STATE/UT|YEAR|Till 3 (night)|3-6 hrs. (Night)|6-9 hrs (Day)|9-12 hrs (Day)|12-15 hrs (Day)|15-18 hrs (Day)|18-21 hrs (Night)|21-24 hrs (Night)|
+---------+----+--------------+----------------+-------------+--------------+---------------+---------------+-----------------+-----------------+
|Karnataka|2001|          2426|            2925|         3775|          5210|           4799|           5131|             4807|             3787|
|Karnataka|2002|          2663|            3482|         4123|          5865|           5414|           5315|             5141|             3658|
|Karnataka|2003|          2719|            3398|         4095|          5735|           5696|           6154|             5687|             4027|
|Karnataka|2004|          2576|            3106|         4140|          5955|           5888|           6400|             63

In [54]:
#Which state has more accidents from year 2001 to 2014?
acc = total.select('STATE/UT','YEAR','Sum').where(col('YEAR') <= 2014)
state = acc.groupBy('STATE/UT','Sum').count().agg({'Sum':'sum'}).collect()[0][0]
state

5599501

In [56]:
#8

highest_acc = df1.select([col for col in df1.columns if col!= 'STATE/UT' and col!='YEAR'])
var  = {}
for i in highest_acc.columns:
  var[i] = highest_acc.agg({i:'max'}).collect()[0][0]
df4 = spark.createDataFrame(var.items(), ["month", "value"])
maxValue = df4.agg({'value':'max'}).collect()[0][0]
df4.select('month','value').where(col('value')==maxValue).show()

+-----+-----+
|month|value|
+-----+-----+
|  MAY| 9330|
+-----+-----+

